In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from nb_005 import *
from collections import Counter

# Wikitext 2

## Data

Download the dataset [here](https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip) and unzip it so it's in the folder wikitext.

In [3]:
EOS = '<eos>'
PATH=Path('data/wikitext')

Small helper function to read the tokens.

In [4]:
def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [5]:
train_tok = read_file('wiki.train.tokens')
valid_tok = read_file('wiki.valid.tokens')
test_tok = read_file('wiki.test.tokens')

In [6]:
len(train_tok), len(valid_tok), len(test_tok)

(36718, 3760, 4358)

In [7]:
' '.join(train_tok[4][:20])

'The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II'

In [8]:
cnt = Counter(word for sent in train_tok for word in sent)
cnt.most_common(10)

[('the', 113161),
 (',', 99913),
 ('.', 73388),
 ('of', 56889),
 ('<unk>', 54625),
 ('and', 50603),
 ('in', 39453),
 ('to', 39190),
 ('<eos>', 36718),
 ('a', 34237)]

Give an id to each token and add the pad token (just in case we need it).

In [9]:
itos = [o for o,c in cnt.most_common()]
itos.insert(0,'<pad>')

In [10]:
vocab_size = len(itos); vocab_size

33279

Creates the mapping from token to id then numericalizing our datasets.

In [11]:
stoi = collections.defaultdict(lambda : 5, {w:i for i,w in enumerate(itos)})

In [12]:
train_ids = np.array([([stoi[w] for w in s]) for s in train_tok])
valid_ids = np.array([([stoi[w] for w in s]) for s in valid_tok])
test_ids = np.array([([stoi[w] for w in s]) for s in test_tok])

In [13]:
class LanguageModelLoader():
    "Creates a dataloader with bptt slightly changing."
    def __init__(self, nums, bs, bptt, backwards=False):
        self.bs,self.bptt,self.backwards = bs,bptt,backwards
        self.data = self.batchify(nums)
        self.first,self.i,self.iter = True,0,0
        self.n = len(self.data)

    def __iter__(self):
        self.i,self.iter = 0,0
        while self.i < self.n-1 and self.iter<len(self):
            if self.first and self.i == 0: self.first,seq_len = False,self.bptt + 25
            else:
                bptt = self.bptt if np.random.random() < 0.95 else self.bptt / 2.
                seq_len = max(5, int(np.random.normal(bptt, 5)))
            res = self.get_batch(self.i, seq_len)
            self.i += seq_len
            self.iter += 1
            yield res

    def __len__(self): return (self.n-1) // self.bptt

    def batchify(self, data):
        nb = data.shape[0] // self.bs
        data = np.array(data[:nb*self.bs]).reshape(self.bs, -1).T
        if self.backwards: data=data[::-1]
        return LongTensor(data)

    def get_batch(self, i, seq_len):
        seq_len = min(seq_len, len(self.data) - 1 - i)
        return self.data[i:i+seq_len], self.data[i+1:i+1+seq_len].contiguous().view(-1)

In [14]:
bs,bptt = 20,10
train_dl = LanguageModelLoader(np.concatenate(train_ids), bs, bptt)
valid_dl = LanguageModelLoader(np.concatenate(valid_ids), bs, bptt)

In [15]:
DataBunch

nb_002b.DataBunch

In [16]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, device=None, dl_tfms=None):
        self.device = default_device if device is None else device
        if not isinstance(train_dl, DeviceDataLoader):
            train_dl = DeviceDataLoader(train_dl, self.device, progress_func=tqdm, tfms=dl_tfms)
        if not isinstance(valid_dl, DeviceDataLoader):
            valid_dl = DeviceDataLoader(valid_dl, self.device, progress_func=tqdm, tfms=dl_tfms)
        self.train_dl,self.valid_dl = train_dl,valid_dl

    @classmethod
    def create(cls, train_ds, valid_ds, train_tfm=None, valid_tfm=None, bs=64, dl_tfms=None, **kwargs):
        train_dl = DeviceDataLoader.create(DatasetTfm(train_ds, train_tfm), bs,   shuffle=True,  tfms=dl_tfms, **kwargs)
        valid_dl = DeviceDataLoader.create(DatasetTfm(valid_ds, valid_tfm), bs*2, shuffle=False, tfms=dl_tfms, **kwargs)
        return cls(train_dl, valid_dl, **kwargs)

    @property
    def train_ds(self): return self.train_dl.dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dl.dataset

In [17]:
data = DataBunch(train_dl, valid_dl)

## Model

### 1. Dropout

We want to use the AWD-LSTM from [Stephen Merity](https://arxiv.org/abs/1708.02182). First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the averga of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)`.

In [18]:
def dropout_mask(x, sz, p):
    "Returns a dropout mask of the same type as x, size sz, with probability p to cancel an element."
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [19]:
x = torch.randn(10,10)
dropout_mask(x, (10,10), 0.5)

tensor([[0., 2., 0., 2., 0., 2., 2., 2., 2., 2.],
        [0., 0., 0., 2., 0., 2., 0., 2., 2., 0.],
        [0., 2., 2., 2., 0., 0., 2., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 0., 2., 2., 0.],
        [0., 0., 2., 0., 2., 0., 2., 0., 2., 2.],
        [2., 2., 2., 2., 2., 0., 2., 0., 0., 2.],
        [2., 2., 2., 0., 2., 2., 0., 0., 0., 2.],
        [2., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 2., 2., 2., 0., 0.],
        [0., 0., 2., 0., 0., 2., 0., 0., 0., 2.]])

Once with have a dropout mask `m`, applying the dropout to `x` is simply done by `x = x * m`. We create our own dropout mask and don't rely on pytorch dropout because we want to nullify the coefficients on the batch dimension but not the token dimension (aka the same coefficients are replaced by zero for each word in the sentence). 

Inside a RNN, a tensor x will have three dimensions: seq_len, bs, vocab_size, so we create a dropout mask for the last two dimensions and broadcast it to the first dimension.

In [20]:
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
        return x.mul_(m)
        #return x * m #returning x.mul_(m) gives an error later ()

In [21]:
dp_test = RNNDropout(0.5)
x = torch.randn(2,5,10)
x, dp_test(x)

(tensor([[[-0.0000, -1.4171,  0.0000, -0.0000, -0.0000,  0.0000, -3.4614,
           -0.0000,  0.0000, -0.0000],
          [ 1.1734, -2.5837, -3.6266,  1.5591,  0.4078,  0.0000, -0.6922,
            0.0000, -0.0000,  0.0000],
          [ 2.4662, -2.7882, -0.7391, -0.0000, -5.1134,  0.0000, -3.8518,
           -0.0000,  0.0000,  0.5909],
          [-0.0000,  0.1294,  0.1673,  0.5515,  0.0000, -0.0000,  0.0000,
            2.6462, -3.0013, -0.4970],
          [-0.9196,  0.0000, -1.2798,  1.6221,  0.0000,  0.2562,  0.3878,
            0.0000,  1.4693, -0.6156]],
 
         [[ 0.0000, -1.7398,  0.0000, -0.0000, -0.0000,  0.0000, -1.9939,
           -0.0000, -0.0000,  0.0000],
          [-0.5702,  1.1506,  0.3808, -1.6470,  2.3314, -0.0000, -2.1978,
           -0.0000,  0.0000, -0.0000],
          [-1.2792,  2.3786, -1.9599,  0.0000, -0.0600,  0.0000,  3.6795,
           -0.0000,  0.0000, -0.2047],
          [ 0.0000,  0.7660, -1.8556,  1.6748, -0.0000,  0.0000,  0.0000,
           -4.0573,

In [22]:
import warnings

In [23]:
class WeightDropout(nn.Module):
    "A module that warps another layer in which some weights will be replaced by 0 during training."
    
    def __init__(self, module, weight_p, layer_names=['weight_hh_l0']):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
    
    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)
            
    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)
    
    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

In [24]:
module = nn.LSTM(20, 20)
dp_module = WeightDropout(module, 0.5)
opt = optim.SGD(dp_module.parameters(), 10)
dp_module.train()

WeightDropout(
  (module): LSTM(20, 20)
)

In [25]:
x = torch.randn(2,5,20)
x.requires_grad_(requires_grad=True)
h = (torch.zeros(1,5,20), torch.zeros(1,5,20))
for _ in range(5): x,h = dp_module(x,h)

In [26]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.0000,  0.3187,  0.0000,  ..., -0.4108,  0.0000,  0.0000],
         [-0.2400,  0.2574, -0.0138,  ..., -0.3312, -0.2832,  0.2427],
         [ 0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         ...,
         [-0.0000,  0.0000, -0.1935,  ...,  0.2950, -0.0000, -0.0000],
         [-0.0000, -0.3660,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000, -0.4122,  ..., -0.0000, -0.0000, -0.3854]],
        grad_fn=<MulBackward0>), Parameter containing:
 tensor([[-0.1905,  0.1593,  0.0876,  ..., -0.2054,  0.0913,  0.0939],
         [-0.1200,  0.1287, -0.0069,  ..., -0.1656, -0.1416,  0.1213],
         [ 0.0281, -0.1436, -0.0415,  ..., -0.1894, -0.0653, -0.0035],
         ...,
         [-0.0255,  0.1765, -0.0967,  ...,  0.1475, -0.0773, -0.0420],
         [-0.1890, -0.1830,  0.0538,  ...,  0.1762, -0.2195,  0.2178],
         [ 0.1789, -0.0877, -0.2061,  ..., -0.0638, -0.0124, -0.1927]],
        requires_grad=True))

In [27]:
target = torch.randint(0,20,(10,)).long()
loss = F.nll_loss(x.view(-1,20), target)
loss.backward()
opt.step()

In [28]:
w, w_raw = getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')
w.grad, w_raw.grad

(None, tensor([[ 0.0000, -0.0003,  0.0001,  ..., -0.0001, -0.0003, -0.0002],
         [-0.0003, -0.0004,  0.0003,  ..., -0.0001, -0.0004, -0.0002],
         [-0.0002, -0.0000,  0.0000,  ..., -0.0001, -0.0005,  0.0000],
         ...,
         [-0.0000,  0.0000, -0.0001,  ...,  0.0001,  0.0000, -0.0000],
         [ 0.0001, -0.0009, -0.0001,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0003,  0.0002,  0.0009,  ...,  0.0000,  0.0001, -0.0006]]))

In [29]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.0000,  0.3187,  0.0000,  ..., -0.4108,  0.0000,  0.0000],
         [-0.2400,  0.2574, -0.0138,  ..., -0.3312, -0.2832,  0.2427],
         [ 0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         ...,
         [-0.0000,  0.0000, -0.1935,  ...,  0.2950, -0.0000, -0.0000],
         [-0.0000, -0.3660,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000, -0.0000, -0.4122,  ..., -0.0000, -0.0000, -0.3854]],
        grad_fn=<MulBackward0>), Parameter containing:
 tensor([[-0.1907,  0.1620,  0.0863,  ..., -0.2042,  0.0946,  0.0958],
         [-0.1167,  0.1323, -0.0104,  ..., -0.1648, -0.1377,  0.1231],
         [ 0.0303, -0.1434, -0.0415,  ..., -0.1883, -0.0602, -0.0035],
         ...,
         [-0.0254,  0.1765, -0.0962,  ...,  0.1467, -0.0773, -0.0416],
         [-0.1899, -0.1737,  0.0553,  ...,  0.1761, -0.2199,  0.2180],
         [ 0.1761, -0.0893, -0.2151,  ..., -0.0642, -0.0137, -0.1864]],
        requires_grad=True))

In [30]:
class EmbeddingDropout(nn.Module):

    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [31]:
enc = nn.Embedding(100,20, padding_idx=0)
enc_dp = EmbeddingDropout(enc, 0.5)

In [32]:
x = torch.randint(0,100,(25,)).long()

In [33]:
enc_dp(x)

tensor([[ 1.0798, -2.1721,  1.0404,  0.0741,  2.8412, -0.0902,  0.3362, -3.1864,
          0.7772, -0.6373,  2.4475, -1.1019, -0.9381, -1.3904,  0.1041, -1.2550,
          3.8392,  0.0197, -1.4873, -1.5699],
        [ 0.0515, -1.1362, -0.7684, -2.4888, -1.0197,  1.7072, -1.2673, -0.6663,
          1.3171,  1.1235, -2.0645,  0.3488,  0.9428, -1.1782,  2.2130, -0.9402,
         -0.0917,  2.3527,  0.7280,  5.6097],
        [ 0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,
          0.0000, -0.0000, -0.0000, -0.0000,  0.0000, -0.0000,  0.0000,  0.0000,
          0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  0.0000,
         -0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000,
          0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000,  0.0000, -0.0000,
      

### 2. AWD-LSTM

In [34]:
def repackage_var(h):
    "Detaches h from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)

In [41]:
class RNNCore(nn.Module):
    "AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182"

    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token, bidir=False,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5, qrnn=False):
        
        super().__init__()
        self.bs,self.qrnn,self.ndir = 1, qrnn,(2 if bidir else 1)
        self.emb_sz,self.n_hid,self.n_layers = emb_sz,n_hid,n_layers
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                                   save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True, 
                                   use_cuda=torch.cuda.is_available()) for l in range(n_layers)]
            if weight_p != 0.:
                for rnn in self.rnns:
                    rnn.linear = WeightDropout(rnn.linear, weight_p, layer_names=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if weight_p != 0.: self.rnns = [WeightDropout(rnn, weight_p) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.encoder_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        self.hidden = repackage_var(new_hidden)
        return raw_outputs, outputs

    def one_hidden(self, l):
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        return self.weights.new(self.ndir, self.bs, nh).zero_()

    def reset(self):
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self.one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]

In [42]:
class LinearDecoder(nn.Module):
    "To go on top of a RNN_Core module"
    
    initrange=0.1
    
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1])
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [43]:
class SequentialRNN(nn.Sequential):
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [44]:
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, tie_weights=True, qrnn=False, bias=True,
                 output_p=0.4, hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
    "To create a full AWD-LSTM"
    rnn_enc = RNNCore(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token, qrnn=qrnn,
                 hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [46]:
tst_model = get_language_model(500, 20, 100, 2, 0, qrnn=True)
tst_model.cuda()

SequentialRNN(
  (0): RNNCore(
    (encoder): Embedding(500, 20, padding_idx=0)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(500, 20, padding_idx=0)
    )
    (rnns): ModuleList(
      (0): QRNNLayer(
        (linear): WeightDropout(
          (module): Linear(in_features=40, out_features=300, bias=True)
        )
      )
      (1): QRNNLayer(
        (linear): WeightDropout(
          (module): Linear(in_features=100, out_features=60, bias=True)
        )
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=20, out_features=500, bias=True)
    (output_dp): RNNDropout()
  )
)

In [47]:
x = torch.randint(0, 500, (10,5)).long()
z = tst_model(x.cuda())

In [48]:
len(z)

3

### 3. Callback to train the model

In [49]:
@dataclass
class RNNTrainer(Callback):
    learn:Learner
    bptt:int
    clip:float=None
    alpha:float=0.
    beta:float=0.
    
    def on_loss_begin(self, last_output, **kwargs):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = last_output[1],last_output[2]
        return last_output[0]
    
    def on_backward_begin(self, last_loss, last_input, last_output, **kwargs):
        #Adjusts the lr to the bptt selected
        self.learn.opt.lr *= last_input.size(0) / self.bptt
        #AR and TAR
        if self.alpha != 0.:  last_loss += (self.alpha * self.out[-1].pow(2).mean()).sum()
        if self.beta != 0.:
            h = self.raw_out[-1]
            if len(h)>1: last_loss += (self.beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
        return last_loss
    
    def on_backward_end(self, **kwargs):
        if self.clip:  nn.utils.clip_grad_norm_(self.learn.model.parameters(), self.clip)

In [50]:
emb_sz, nh, nl = 400, 1150, 3
model = get_language_model(vocab_size, emb_sz, nh, nl, 0, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)
learn = Learner(data, model)

C:\Users\Sylvain\Anaconda3\envs\fastai1\lib\site-packages\torch\backends\cudnn\__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [51]:
learn.opt_fn = partial(optim.Adam, betas=(0.8,0.99))
learn.callbacks.append(RNNTrainer(learn, bptt, clip=0.12, alpha=2, beta=1))

In [52]:
fit_one_cycle(learn, 5e-3, 1, (0.8,0.7), wd=1.2e-6)

KeyboardInterrupt: 